Now that we have clean data we can train a model. The user will give a car (manufacturer, model and year) with a certain odometer reading and the price that the seller is asking for. The model will look at the data available for that car, fit a linear regression and look at the predicted price for the vehicle. Then we can tell the user whether the price is over, under or around what is typical. If the price is under then this suggests they may be getting a good deal.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
from tqdm.notebook import tqdm
sys.path.append("..")
from car_purchase_help.data_processing import format_raw_df, split_by_description
from car_purchase_help.model1 import predict_price, fit_lin_regression, get_advice

%matplotlib inline
pd.options.display.float_format = '{:20,.2f}'.format
plt.style.use('fivethirtyeight')

In [2]:
# https://www.kaggle.com/austinreese/craigslist-carstrucks-data/
df = pd.read_csv(Path('../data/vehicles.csv'))
df = format_raw_df(df)

We loop over every manufacturer, then model and then year training a regression and saving in a pkl file for quick access by the web app.
There are various errors that can occur which are caught and ignored.

In [3]:
for manu in tqdm(df['manufacturer'].unique()):
    data_manu = df[df['manufacturer'] == manu]
    for mod in tqdm(data_manu['model'].unique()):
        data_mod = data_manu[data_manu['model'] == mod]
        for year in data_mod['year'].unique():
            try:
                fit_lin_regression(df, manu, mod, year)
            except Exception as e:
                pass

Now that the models are saved we can make predicitons.

Let's see what advice we get for a Honda Accord from 2015 with 100,000 miles on the clock.

In [15]:
pred_price, mean_abs_res = predict_price('honda', 'accord', 2015, odometer=100000)

In [16]:
get_advice(pred_price, listed_price=6000, mean_absolute_residual=mean_absolute_residual)

'This appears to be a very good deal'

In [17]:
get_advice(pred_price, listed_price=10000, mean_absolute_residual=mean_absolute_residual)

'This appears to be a good deal'

In [18]:
get_advice(pred_price, listed_price=14000, mean_absolute_residual=mean_absolute_residual)

'This appears to be a bad deal'

In [19]:
get_advice(pred_price, listed_price=18000, mean_absolute_residual=mean_absolute_residual)

'This appears to be a very bad deal'

In [20]:
pred_price, mean_abs_res = predict_price('tesla', 'model-s', 2015, odometer=10000)

AssertionError: No regression model for this car from that year

This query fails because there is too little data for Teslas and no model was saved

In [21]:
pred_price, mean_abs_res = predict_price(' MerCedeS-BenZ ', 'C-class', 2015, 45000.4)

In [23]:
get_advice(pred_price, listed_price=20000, mean_absolute_residual=mean_abs_res)

'This appears to be a fair price for the car'

In [24]:
get_advice(pred_price, listed_price=30000, mean_absolute_residual=mean_abs_res)

'This appears to be a very bad deal'

It appears this model is giving appropriate predictions and is ready to be used in the web app.